In [1]:
!pip install pandas

In [29]:
import pandas as pd

export_df = pd.read_csv("tea_export_data.csv")

In [30]:
export_df.head(10)

,Year,Category,Package Type,Quantity (Kg),Price (Rs.)
0,2012-01,Black,Bags,1716666.0,932.61
1,2012-01,NaN,Bulk,9403630.0,446.65
2,2012-01,NaN,Packets (4g - 1kg),6463463.0,481.46
3,2012-01,NaN,Packets (1kg - 3kg),54860.0,626.19
4,2012-01,NaN,Packets (3kg - 5kg),985540.0,427.39
5,2012-01,NaN,Packets (5kg - 10kg),2907908.0,422.63
6,2012-01,NaN,Sub Total,21532067.0,492.17
7,2012-01,Green,Bags,52148.0,1431.36
8,2012-01,NaN,Bulk,30210.0,522.10
9,2012-01,NaN,Packets (4g - 1kg),59321.0,906.74


In [31]:
# Filter rows where 'Package Type' is 'Total Exports (Without RTD)'
filtered_export_df = export_df[export_df['Package Type'].isin(['Total Exports (Without RTD)', 'Total Exports'])]

# Group by 'Year' and calculate sum for 'Quantity' and 'Price'
processed_export_df = filtered_export_df.groupby('Year', as_index=False).agg({
    'Quantity (Kg)': 'sum',
    'Price (Rs.)': 'sum'
})

# Rename columns for clarity
processed_export_df.rename(columns={
    'Year': 'Year-Month',
    'Quantity (Kg)': 'Export_Quantity_(Kg)',
    'Price (Rs.)': 'Export_Price_(Rs.)'
}, inplace=True)

In [32]:
# Save the processed dataset to a new CSV file (optional)
processed_export_df.to_csv("processed_tea_export_data.csv", index=False)

In [33]:
processed_export_df.head(10)

,Year-Month,Export_Quantity_(Kg),Export_Price_(Rs.)
0,2012-01,21824945.0,497.66
1,2012-02,22448624.0,509.15
2,2012-03,27702254.0,536.71
3,2012-04,23290994.0,540.91
4,2012-05,30004885.0,567.32
5,2012-06,22804265.0,553.03
6,2012-07,25761299.0,555.55
7,2012-08,21064283.0,561.75
8,2012-09,30620887.0,570.06
9,2012-10,24246574.0,576.88


In [34]:
prod_df = pd.read_csv("tea_production_data.csv")

prod_df.head(10)

,Year,Elevation,Quantity (Kg)
0,2012-01,High,5361185.05
1,2012-01,Medium,3652353.15
2,2012-01,Low,14348315.90
3,2012-01,Total,23361854.10
4,2012-02,Total,22627358.06
5,2012-02,Low,13133213.00
6,2012-02,Medium,3955745.75
7,2012-02,High,5538399.31
8,2012-03,High,6385525.09
9,2012-03,Medium,4625138.90


In [35]:
# Pivot the table to create separate columns for each elevation
pivoted_prod_df = prod_df.pivot(index='Year', columns='Elevation', values='Quantity (Kg)')

# Ensure the order of columns and fill missing values with 0 (if any)
pivoted_prod_df = pivoted_prod_df[['High', 'Medium', 'Low']].fillna(0)

# Reset index to make 'Year' a column again
processed_prod_df = pivoted_prod_df.reset_index()

In [39]:
processed_prod_df.head(10)

Elevation,Year-Month,High_Grown_Quantity_(Kg),Medium_Grown_Quantity_(Kg),Low_Grown_Quantity_(Kg)
0,2012-01,5361185.05,3652353.15,14348315.90
1,2012-02,5538399.31,3955745.75,13133213.00
2,2012-03,6385525.09,4625138.90,17929328.00
3,2012-04,7323011.00,5748401.20,16982493.50
4,2012-05,8726319.70,5898297.10,18086627.25
5,2012-06,5963710.45,3842289.70,16737376.00
6,2012-07,4064094.35,3429525.60,17506502.00
7,2012-08,5372381.95,4358458.00,16139942.00
8,2012-09,4906061.70,3517426.01,19011266.10
9,2012-10,5860294.50,4052676.95,17499597.00


In [37]:
# Rename columns for clarity
processed_prod_df.rename(columns={
    'Year': 'Year-Month',
    'High': 'High_Grown_Quantity_(Kg)',
    'Medium': 'Medium_Grown_Quantity_(Kg)',
    'Low': 'Low_Grown_Quantity_(Kg)',
}, inplace=True)

In [38]:
processed_prod_df.to_csv("processed_tea_production_data.csv", index=False)

In [40]:
rate_df = pd.read_csv("exchange_rates_data.csv")

In [46]:
# Convert 'Date' to datetime format
rate_df['Date'] = pd.to_datetime(rate_df['Date'])

# Sort the data by 'Date' in ascending order
rate_df.sort_values(by='Date', inplace=True)

# Create a 'Year-Month' column
rate_df['Year-Month'] = rate_df['Date'].dt.to_period('M').astype(str)

# Calculate the average rate for each row
rate_df['Avg_Rate'] = rate_df[['Buy Rate', 'Sell Rate']].mean(axis=1)

# Group by 'Year-Month' and calculate the monthly average
monthly_avg = rate_df.groupby('Year-Month', as_index=False).agg({
    'Avg_Rate': 'mean'
}).rename(columns={'Avg_Rate': 'Avg_Exchange_Rate'})

In [48]:
# Generate a complete range of Year-Month values from the dataset's date range
date_range = pd.date_range(start=rate_df['Date'].min(), end=rate_df['Date'].max(), freq='ME')
all_months = pd.DataFrame({'Year-Month': date_range.to_period('M').astype(str)})

# Merge the generated months with the monthly averages
full_rate_data = pd.merge(all_months, monthly_avg, on='Year-Month', how='left')

# Fill missing values in 'Avg_Exchange_Rate' with 200.00
full_rate_data['Avg_Exchange_Rate'] = full_rate_data['Avg_Exchange_Rate'].fillna(200.00)

In [49]:
full_rate_data.head(10)

,Year-Month,Avg_Exchange_Rate
0,2011-01,110.986753
1,2011-02,110.356467
2,2011-03,110.453504
3,2011-04,110.285420
4,2011-05,109.842925
5,2011-06,109.667083
6,2011-07,109.522410
7,2011-08,109.480400
8,2011-09,200.000000
9,2011-10,200.000000


In [51]:
full_rate_data.to_csv("processed_exchange_rate_data.csv", index=False)